圖片來源:https://data.mendeley.com/datasets/7w7fccy7ky/4

In [28]:
import os
import cv2
from ultralytics import YOLO
import csv

conf = 0.7
dir = "datanew1.v2i.multiclass"
dir2 = "train"
# Path to the directory containing images
input_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}'  # Directory containing images
output_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}_detect_{conf}_315'  # Output directory for annotated images
output_file = f"C:/Users/User/Desktop/dataset/{dir}/{dir2}_keypoints_{conf}_315.csv"  # Output CSV file for keypoints data

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model

def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for j in range(17):  # Assuming there are 17 keypoints
            x, y = r.keypoints.xyn[i][j].numpy()
            combined_results.append((x, y))
    box = results[0].boxes.xywhn.numpy().reshape(-1, 2)
    list_of_tuples = [tuple(row) for row in box]
    combined_results.extend(list_of_tuples)
    return combined_results

# Process each PNG image in the input directory
all_keypoints_data = []
# Get all files in input_dir
all_files = os.listdir(input_dir)
# Filter out only files that end with .png and are valid files
image_files = [filename for filename in all_files if os.path.isfile(os.path.join(input_dir, filename)) and filename.lower().endswith('.jpg')]
# Sort the filtered image files
image_files.sort()

for idx, filename in enumerate(image_files):
    input_path = os.path.join(input_dir, filename)
    
    # Output path remains the same as input path, with '_detect_confidence' appended
    output_path = os.path.join(output_dir, filename)

    # Load the image
    image = cv2.imread(input_path)
    # flip
    # image = cv2.flip(image, 0)
    # Get the dimensions of the image
    (h, w) = image.shape[:2]
    
    # Calculate the center of the image
    center = (w // 2, h // 2)
    
    # Create the rotation matrix for 45 degrees
    angle = 315
    scale = 1.0
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    
    # Perform the affine transformation (rotation)
    image = cv2.warpAffine(image, rotation_matrix, (w, h))
    # Perform pose detection
    results = model(image, conf=conf)

    if len(results[0].boxes) == 1:
        # Draw the results on the image
        annotated_image = results[0].plot()
        # Save the annotated image with the original filename
        cv2.imwrite(output_path, annotated_image)
        print(f"Processed and saved: {output_path}")
        keypoints_data = process_image(results)
        keypoints_data.insert(0, filename)  # Use the original filename
        all_keypoints_data.append(keypoints_data)
    else:
        keypoints_data = [filename] + [None] * 19  # Assuming 17 keypoints with x,y pairs
        all_keypoints_data.append(keypoints_data)

# Write keypoints data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}" for i in range(17)] + ["xb, yb"] + ["w, h"]  # Assuming 17 keypoints with x,y pairs
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)



0: 640x640 1 person, 151.7ms
Speed: 3.0ms preprocess, 151.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/train_detect_0.7_315\0_jpg.rf.273ffe056a0f59ed06a2e3bb671e36dd.jpg

0: 640x640 1 person, 143.8ms
Speed: 4.1ms preprocess, 143.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/train_detect_0.7_315\0_jpg.rf.279a431bf6f08ce5445372e2a05dd189.jpg

0: 640x640 (no detections), 145.5ms
Speed: 3.0ms preprocess, 145.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 141.7ms
Speed: 2.0ms preprocess, 141.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/train_detect_0.7_315\0_jpg.rf.626cd65bd869b77586b8686c89c1efcd.jpg

0: 640x640 1 person, 141.3ms
Speed: 4.0ms preprocess, 

In [19]:
import os
import pandas as pd
s = 4
# Specify the directory containing the CSV files
directory = f'D:/Subject.{s}'

# List all files in the directory
files = os.listdir(directory)

# Filter the list to include only CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file into a DataFrame and append it to the list
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames along axis 0
combined_df = pd.concat(dataframes, axis=0)

# Reset the index of the combined DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
print(combined_df)

# Specify the path for the output CSV file
output_file = f'D:/combined_Subject_{s}.csv'

# Export the combined DataFrame to a CSV file
combined_df.to_csv(output_file, index=False)


             image_name        x1, y1, confidence1      x2, y2, confidence2  \
0     detected_0000.png                 (0.0, 0.0)               (0.0, 0.0)   
1     detected_0002.png                 (0.0, 0.0)               (0.0, 0.0)   
2     detected_0004.png                 (0.0, 0.0)               (0.0, 0.0)   
3     detected_0006.png                 (0.0, 0.0)               (0.0, 0.0)   
4     detected_0008.png                 (0.0, 0.0)               (0.0, 0.0)   
...                 ...                        ...                      ...   
1628  detected_0472.png   (0.55212075, 0.07922764)  (0.5568776, 0.06957651)   
1629  detected_0474.png    (0.5495374, 0.07559716)  (0.5553281, 0.06585138)   
1630  detected_0476.png    (0.5505092, 0.07821099)  (0.5555378, 0.06897019)   
1631  detected_0478.png  (0.54478145, 0.074853435)  (0.5509016, 0.06510606)   
1632  detected_0480.png   (0.54442215, 0.07527539)  (0.550184, 0.065937564)   

           x3, y3, confidence3        x4, y4, confi